In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import math

In [15]:
train = pd.read_csv('train.csv')
#test = pd.read_csv('test.csv')
validation = pd.read_csv('new_validation.csv')

In [73]:
XG = pd.read_csv('XGboosting_pCTR.csv')

In [18]:
validation_o =  pd.read_csv('validation.csv')

In [3]:
Ada = pd.read_csv('Ada_validation.csv')

In [46]:
# c = 6
# lambdas = 0.000025

In [40]:
c =  np.linspace(6,6 , 1)
lambdas = np.linspace(0.000025,  0.0000025, 1)

In [41]:
range_m = np.linspace(100, 150, 10) 

In [42]:
range_n =np.linspace(2, 6, 10) 

In [17]:
basebids = np.linspace(20, 350, 100) 

In [6]:
avgCTR = 0.0007375623256619447

<font face="微软雅黑" size=6 color=#000000> Squared Bidding

In [4]:
def Non_LinearStrategy(pCTR, base_bid,avgCTR,m,n):
    avgCTR = np.repeat(avgCTR*100, pCTR.shape[0])
    base_bid =  np.repeat(base_bid, pCTR.shape[0])
    m= np.repeat(m, pCTR.shape[0])
    n= np.repeat(n, pCTR.shape[0])
    bidprices  =  base_bid  * ((pCTR['pCTR']/ avgCTR)**2) + m* (pCTR['pCTR']/ avgCTR) + n
    return bidprices

<font face="微软雅黑" size=6 color=#000000> Expoential Bidding 

In [9]:
def Exp(pCTR, base_bid,avgCTR):
    avgCTR = np.repeat(avgCTR*100, pCTR.shape[0])
    base_bid =  np.repeat(base_bid, pCTR.shape[0])
    bidprices  = base_bid  * np.exp(pCTR['pCTR']/ avgCTR)
    return bidprices

In [149]:
#lambda_range =np.linspaces (math.exp(-10),     )

<font face="微软雅黑" size=6 color=#000000> ORTB1

In [4]:

def Optimal_RealTime_Bidding_1(pCTR, c, lambdas):
    c= np.repeat(c, pCTR.shape[0])
    lambdas = np.repeat(lambdas, pCTR.shape[0])
    bidprices = np.sqrt( (c/lambdas * pCTR['pCTR'] )+c**2) - c
    return bidprices

<font face="微软雅黑" size=6 color=#000000> ORTB2

In [25]:

def Optimal_RealTime_Bidding_2(pCTR, c, lambdas):
    c= np.repeat(c, pCTR.shape[0])
    lambdas = np.repeat(lambdas, pCTR.shape[0])
    a =  ((pCTR['pCTR'] + np.sqrt( (c**2)*(lambdas**2) + lambdas**2))/(c*lambdas))**(1/3)  
    b = ((c*lambdas)/ (pCTR['pCTR'] + np.sqrt((c**2)*(lambdas**2) + lambdas**2)))**(1/3)  
    bidprices =c*(a - b)
    return bidprices

<font face="微软雅黑" size=6 color=#000000> Strategy Evaluation 

In [6]:
def Evaluate(validation, pre_bidprices):
    #test if win
    bidprice = pre_bidprices
    budget = 6250000
    click = 0
    impression = 0
    spend = 0

    for i in range(len(validation)):

        if bidprice[i] > validation['payprice'][i] and budget > validation['payprice'][i]:

            click = click + validation['click'][i]
            impression = impression + 1
            spend = spend +  validation['payprice'][i]
            budget = budget - validation['payprice'][i]


    ctr =  np.round(click/impression*100,5)   
    cpm = np.round(spend/impression*1000,5) 
    cpc = np.round(spend/click,2)

    return [impression,click, ctr, spend, cpm, cpc]  

<font face="微软雅黑" size=6 color=#000000> Tuning Parameter for Square 

In [ ]:
k=0
results  = pd.DataFrame(columns = ["basebid","impression","click","ctr","spend","cpm","cpc","m","n"])     

for basebid in basebids:
    for m in range_m:
        for n in range_n:
            pre_bidprices = Non_LinearStrategy(XG, basebid,avgCTR,m,n)
            impression, click, ctr, spend, cpm, cpc = Evaluate(validation, pre_bidprices)
            results.loc[k]  = np.round(basebid,5) , impression, click, ctr, spend, cpm, cpc, m, n
            k = k+1

In [ ]:
results.click.max()

In [ ]:
best = results.loc[(results['click'] == 171)]

<font face="微软雅黑" size=6 color=#000000> Tuning Parameter for Exponential

In [18]:
k=0
results  = pd.DataFrame(columns = ["basebid","impression","click","ctr","spend","cpm","cpc"])     

for basebid in basebids:
    pre_bidprices = Exp(XG, basebid,avgCTR)
    impression, click, ctr, spend, cpm, cpc = Evaluate(validation, pre_bidprices)
    results.loc[k]  = np.round(basebid,5) , impression, click, ctr, spend, cpm, cpc
    k = k+1

In [19]:
results.click.max()

161.0

In [20]:
results

,basebid,impression,click,ctr,spend,cpm,cpc
0,20.00000,76263.0,124.0,0.16260,2079856.0,27272.15032,16773.03
1,23.33333,86822.0,129.0,0.14858,2519716.0,29021.63046,19532.68
2,26.66667,95980.0,132.0,0.13753,2967437.0,30917.24318,22480.58
3,30.00000,105449.0,139.0,0.13182,3451408.0,32730.59014,24830.27
4,33.33333,114634.0,145.0,0.12649,3962357.0,34565.28604,27326.60
5,36.66667,124218.0,155.0,0.12478,4525763.0,36434.03532,29198.47
6,40.00000,134894.0,157.0,0.11639,5171415.0,38336.87933,32938.95
7,43.33333,145988.0,161.0,0.11028,5859432.0,40136.39477,36393.99
8,46.66667,149173.0,147.0,0.09854,6249999.0,41897.65574,42517.00
9,50.00000,143231.0,140.0,0.09774,6249999.0,43635.79812,44642.85


In [21]:
best = results.loc[(results['click'] == 161)]

In [22]:
best

,basebid,impression,click,ctr,spend,cpm,cpc
7,43.33333,145988.0,161.0,0.11028,5859432.0,40136.39477,36393.99


<font face="微软雅黑" size=6 color=#000000> Tuning Parameter for ORTB1

In [53]:
# c =6 , lambda = 0.000025
c =  np.linspace(1, 10, 10)
lambdas = np.linspace(0.000024,  0.000024, 1)

In [54]:
k=0
results  = pd.DataFrame(columns = ["impression","click","ctr","spend","cpm","cpc","c","lambdas"])     

for pram_c in c:
    for lamb in lambdas:
        pre_bidprices = Optimal_RealTime_Bidding_1(XG, pram_c, lamb)
        impression, click, ctr, spend, cpm, cpc = Evaluate(validation, pre_bidprices)
        results.loc[k]  = impression, click, ctr, spend, cpm, cpc, pram_c, lamb
        k = k+1

In [55]:
results.click.max()

163.0

In [56]:
results

,impression,click,ctr,spend,cpm,cpc,c,lambdas
0,61454.0,97.0,0.15784,1366459.0,22235.47694,14087.21,1.0,0.000024
1,91550.0,123.0,0.13435,2670193.0,29166.49918,21708.89,2.0,0.000024
2,111722.0,140.0,0.12531,3766379.0,33712.06208,26902.71,3.0,0.000024
3,126950.0,145.0,0.11422,4718811.0,37170.62623,32543.52,4.0,0.000024
4,139087.0,154.0,0.11072,5541850.0,39844.48583,35986.04,5.0,0.000024
5,148827.0,163.0,0.10952,6233976.0,41887.39946,38245.25,6.0,0.000024
6,142755.0,145.0,0.10157,6249999.0,43781.29663,43103.44,7.0,0.000024
7,137834.0,140.0,0.10157,6249999.0,45344.39253,44642.85,8.0,0.000024
8,134042.0,134.0,0.09997,6249999.0,46627.16910,46641.78,9.0,0.000024
9,130601.0,127.0,0.09724,6249999.0,47855.67492,49212.59,10.0,0.000024


<font face="微软雅黑" size=6 color=#000000> Tuning Parameter for ORTB2

In [90]:
# c =6, lambda = 0.0000024
c =  np.linspace(6, 6 , 1)
lambdas = np.linspace(0.0000020,  0.0000030, 10)

In [91]:
k=0
results  = pd.DataFrame(columns = ["impression","click","ctr","spend","cpm","cpc","c","lambdas"])     

for pram_c in c:
    for lamb in lambdas:
        pre_bidprices = Optimal_RealTime_Bidding_2(XG, pram_c, lamb)
        impression, click, ctr, spend, cpm, cpc = Evaluate(validation, pre_bidprices)
        results.loc[k]  = impression, click, ctr, spend, cpm, cpc, pram_c, lamb
        k = k+1

In [92]:
results.click.max()

148.0

In [95]:
best = results.loc[(results['click'] == 148)] 

In [96]:
best

,impression,click,ctr,spend,cpm,cpc,c,lambdas
3,156887.0,148.0,0.09434,6129515.0,39069.61699,41415.64,6.0,0.000002
4,154403.0,148.0,0.09585,5957744.0,38585.67515,40255.03,6.0,0.000002
5,152116.0,148.0,0.09729,5800513.0,38132.16887,39192.66,6.0,0.000003
6,149874.0,148.0,0.09875,5647812.0,37683.73434,38160.89,6.0,0.000003


<font face="微软雅黑" size=6 color=#000000> run on validation 

In [12]:
bidprices = Optimal_RealTime_Bidding_1(XG,6,0.000025)

In [20]:
Group_17 = pd.DataFrame(data = list(zip(validation_o["bidid"], bidprices)),
                               columns = ["bidid", "bidprice"])

In [22]:
Group_17.to_csv("Group_17.csv")